## Project: Predicting Stock Market
## Part 1: Data preparation, model estimation
**Data:** S&P500 index valuation, New York Stock Exchange (NYSE), 1950 to 2015  
**DataQuest Lesson:** https://app.dataquest.io/c/11/m/65/guided-project%3A-predicting-the-stock-market/1/the-dataset?path=2&slug=data-scientist&version=1.2  
**Source:** https://github.com/NickyThreeNames/DataquestGuidedProjects/blob/master/Guided%20Project-%20Predicting%20the%20stock%20market/sphist.csv


#### Useful links

Inspiration notebooks:
* https://medium.com/shiyan-boxer/s-p-500-stock-price-prediction-using-machine-learning-and-deep-learning-328b1839d1b6
* https://medium.com/mlearning-ai/predict-sp500-stock-price-with-python-machine-learning-sentiment-analysis-a296dc276353
* https://medium.com/p/59b06de25357
* https://iopscience.iop.org/article/10.1088/1742-6596/1366/1/012130
* https://towardsdatascience.com/stock-market-predictions-with-rnn-using-daily-market-variables-6f928c867fd2
* https://www.kaggle.com/code/samaxtech/predicting-s-p500-index-linearreg-randomforests
* https://www.kaggle.com/code/janiobachmann/s-p-500-time-series-forecasting-with-prophet

Feature Engineering:
* https://towardsdatascience.com/one-hot-encoding-scikit-vs-pandas-2133775567b8
* https://towardsdatascience.com/using-columntransformer-to-combine-data-processing-steps-af383f7d5260
* https://www.kaggle.com/code/samaxtech/predicting-s-p500-index-linearreg-randomforests
* https://www.alpharithms.com/predicting-stock-prices-with-linear-regression-214618/
* https://towardsdatascience.com/3-basic-steps-of-stock-market-analysis-in-python-917787012143
* https://tradewithpython.com/generating-buy-sell-signals-using-python
* https://not-satoshi.com/course-binance-api-trading-bot-sma-macd-and-bollinger-bands/
* https://technical-analysis-library-in-python.readthedocs.io/en/latest/index.html


To do:
* https://pub.towardsai.net/from-experiments-to-deployment-mlflow-101-part-02-f386022afdc6 ML-Flow


Correlation for non-numeric data (Dython library):  

Variable standarization:  

Variable selection:  

Optimal Binning and WoE transformation for continuous dependent variable:

Diagnostics:

Regression:


Replace in selection:

    Select text region with mouse or keyboard.
    Ctrl+H to toggle find and replace
    Alt+L to toggle find in selection
    Ctrl+Alt+Enter to replace all (or enter to replace individually)


## Analysis plan
1. Import data
2. Research about approach
3. Clean Data
4. Feature Engineering
5. Feature Selection
6. Model training & tuning
7. Final model selection - evaluation on test data
8. Diagnostics


Additional : comparing simple train/test split forecast vs. rolling X-days ? Arima/Sarima/GARCH models vs linear / neural


### 1. Setting up environment

#### 1.1 Importing packages & setting-up parameters

In [69]:
# Set-up auto-reload functions for faster debugging 
# (automatically refreshes changes in subpackages codes)
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# Import parent directory (main project directory)
# for packages importing
import sys
import os

# Getting the parent directory name in which your script is running
parent = os.path.dirname(os.path.realpath(''))

# adding the parent directory to
# the sys.path.
sys.path.append(parent)

# now we can import the module in the parent
# directory.

In [2]:
print(parent)

C:\Users\tgusc\Documents\GitHub\python-gp24


In [71]:
# Project packages import
import gp24package.data.make_dataset as gp24md
import gp24package.features.build_features as gp24feat
# import gp24package.explore_visualise.eda as gp23eda
# import gp24package.features.build_features as gp23feat
# import gp24package.models.hyperparameters_model as gp23hyperparam
# import gp24package.models.train_model as gp23train


# Pylance highligting package issue (not to be worried about)
# https://github.com/microsoft/pylance-release/blob/main/TROUBLESHOOTING.md#unresolved-import-warnings

# Standard Python libraries import
from IPython.display import display, HTML #  tidied-up display
from time import time #  project timer
from itertools import chain # for list iterations
from datetime import datetime
import pandas as pd
import numpy as np

# plots
import matplotlib.pyplot as plt
import seaborn as sn

# Necessary packages
import gp24package
import session_info # build and requirements.txt
import pickle # dump models

import ta #technical-analysis

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

#turning on plot display in JN
%matplotlib inline 
# Setting pandas display options
pd.options.display.max_columns = 300
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 250
pd.options.display.float_format = '{:.4f}'.format

In [72]:
# MANUAL_INPUT - mark of sections of code, that are not automated and have to be manually re-coded to new datasets.

# parameters
seed = 12345
target = 'SalePrice'
ID_vars = ['Order', 'PID']

# List of days - input to VarDescriptiveStatistics class
# based on list of contents we are creating new variables for last X days
# mean, std, min, max 
days = [3, 5, 30, 60, 90, 120, 240, 365]

# One Hot Encoding list
# contains varaibles nad possible values for one hot encoding
categories = [('month',[1,2,3,4,5,6,7,8,9,10,11,12]),
              ('weekday',[0,1,2,3,4,5,6])]

In [73]:
def display_html(data_in, n = 5):
    print(data_in.shape)
    display(HTML(data_in.head(n).to_html()))

#### 1.2 Starting project timer and exporting requirements

In [74]:
# Starting project timer
tic_all = time()

In [75]:
# Collecting packages info and saving to requirements.txt file
session_info.show(cpu = True, std_lib = True, dependencies = True, write_req_file = True,
                  req_file_name = 'requirements.txt')

#### 1.3 Importing and inspecting source data

In [76]:
data = gp24md.MakeDataset('sphist.csv').data
print(type(data))

display(HTML(data.head().to_html()))

<class 'pandas.core.frame.DataFrame'>


,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.4199,2090.4199,2066.7800,2077.0701,4043820000.0000,2077.0701
1,2015-12-04,2051.2400,2093.8401,2051.2400,2091.6899,4214910000.0000,2091.6899
2,2015-12-03,2080.7100,2085.0000,2042.3500,2049.6201,4306490000.0000,2049.6201
3,2015-12-02,2101.7100,2104.2700,2077.1101,2079.5100,3950640000.0000,2079.5100
4,2015-12-01,2082.9299,2103.3701,2082.9299,2102.6299,3712120000.0000,2102.6299


In [77]:
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values(ascending = True, inplace  = True, by = 'Date')
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 16590 entries, 16589 to 0
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16590 non-null  datetime64[ns]
 1   Open       16590 non-null  float64       
 2   High       16590 non-null  float64       
 3   Low        16590 non-null  float64       
 4   Close      16590 non-null  float64       
 5   Volume     16590 non-null  float64       
 6   Adj Close  16590 non-null  float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 1.0 MB
None


### 2. Feature Engineering - without target data information leakage risk.

For purpose of forecasting we should ideally first split the dataset into train/valid and test dataset. This will not be the case in those project. Why? Because we plan to calculate a lot of rolling metrics looking back as far as 365 days (1 year). So in order to prevent later troubles with calculating those metrics on test dataset I am allowing myself to relax this condition and split Feature Engineering part into two parts - without risk of target data information leakage risk and with risk of target data information leakage (like variable transformations based on variable distribution).

#### 2.1 Date features

In [78]:
# Separating date to month, day and weekday variable. Transforming month and weekday variables to a cirle representation to investigate cyclical seasonality.
data2 = gp24feat.DateFeatures(data_in = data, variable = 'Date').output()
data2.head()

,Date,Open,High,Low,Close,Volume,Adj Close,month,day,weekday,month_sin,month_cos,weekday_sin,weekday_cos
16589,1950-01-03,16.6600,16.6600,16.6600,16.6600,1260000.0000,16.6600,1,3,1,0.5000,0.8660,0.7818,0.6235
16588,1950-01-04,16.8500,16.8500,16.8500,16.8500,1890000.0000,16.8500,1,4,2,0.5000,0.8660,0.9749,-0.2225
16587,1950-01-05,16.9300,16.9300,16.9300,16.9300,2550000.0000,16.9300,1,5,3,0.5000,0.8660,0.4339,-0.9010
16586,1950-01-06,16.9800,16.9800,16.9800,16.9800,2010000.0000,16.9800,1,6,4,0.5000,0.8660,-0.4339,-0.9010
16585,1950-01-09,17.0800,17.0800,17.0800,17.0800,2520000.0000,17.0800,1,9,0,0.5000,0.8660,0.0000,1.0000


In [79]:
# Using OneHotEncoder from sklearn to create binary variables representing months and weekdays
data3 = gp24feat.FeaturesOneHotEncoding(data_in = data2, features_list = categories).transform(data2)
data3.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
16589,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
16588,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
16587,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
16586,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
16585,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [80]:
# Concatenate to original data
data4 = pd.concat([data2, data3], axis = 1)
data4.head()

,Date,Open,High,Low,Close,Volume,Adj Close,month,day,weekday,month_sin,month_cos,weekday_sin,weekday_cos,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
16589,1950-01-03,16.6600,16.6600,16.6600,16.6600,1260000.0000,16.6600,1,3,1,0.5000,0.8660,0.7818,0.6235,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
16588,1950-01-04,16.8500,16.8500,16.8500,16.8500,1890000.0000,16.8500,1,4,2,0.5000,0.8660,0.9749,-0.2225,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
16587,1950-01-05,16.9300,16.9300,16.9300,16.9300,2550000.0000,16.9300,1,5,3,0.5000,0.8660,0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
16586,1950-01-06,16.9800,16.9800,16.9800,16.9800,2010000.0000,16.9800,1,6,4,0.5000,0.8660,-0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
16585,1950-01-09,17.0800,17.0800,17.0800,17.0800,2520000.0000,17.0800,1,9,0,0.5000,0.8660,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


#### 2.1 Descriptive statistics and rolling metrics
Calculating last X days min, max, stdev and mean for Closing price and calculated daily return

In [81]:
data5 = gp24feat.VarDescriptiveStatistics(data_in = data4, days = days, var_name = 'Close').output()
display_html(data5)

(16590, 576)


,Close_mean_3,Close_stdev_3,Close_max_3,Close_min_3,Close_mean_5,Close_stdev_5,Close_max_5,Close_min_5,Close_mean_30,Close_stdev_30,Close_max_30,Close_min_30,Close_mean_60,Close_stdev_60,Close_max_60,Close_min_60,Close_mean_90,Close_stdev_90,Close_max_90,Close_min_90,Close_mean_120,Close_stdev_120,Close_max_120,Close_min_120,Close_mean_240,Close_stdev_240,Close_max_240,Close_min_240,Close_mean_365,Close_stdev_365,Close_max_365,Close_min_365,Close_mean_3_stdev_3_ratio,Close_mean_3_max_3_ratio,Close_mean_3_min_3_ratio,Close_stdev_3_mean_3_ratio,Close_stdev_3_max_3_ratio,Close_stdev_3_min_3_ratio,Close_max_3_mean_3_ratio,Close_max_3_stdev_3_ratio,Close_max_3_min_3_ratio,Close_min_3_mean_3_ratio,Close_min_3_stdev_3_ratio,Close_min_3_max_3_ratio,Close_mean_3_stdev_5_ratio,Close_mean_3_max_5_ratio,Close_mean_3_min_5_ratio,Close_stdev_3_mean_5_ratio,Close_stdev_3_max_5_ratio,Close_stdev_3_min_5_ratio,Close_max_3_mean_5_ratio,Close_max_3_stdev_5_ratio,Close_max_3_min_5_ratio,Close_min_3_mean_5_ratio,Close_min_3_stdev_5_ratio,Close_min_3_max_5_ratio,Close_mean_3_mean_5_ratio,Close_stdev_3_stdev_5_ratio,Close_max_3_max_5_ratio,Close_min_3_min_5_ratio,Close_mean_3_stdev_30_ratio,Close_mean_3_max_30_ratio,Close_mean_3_min_30_ratio,Close_stdev_3_mean_30_ratio,Close_stdev_3_max_30_ratio,Close_stdev_3_min_30_ratio,Close_max_3_mean_30_ratio,Close_max_3_stdev_30_ratio,Close_max_3_min_30_ratio,Close_min_3_mean_30_ratio,Close_min_3_stdev_30_ratio,Close_min_3_max_30_ratio,Close_mean_3_mean_30_ratio,Close_stdev_3_stdev_30_ratio,Close_max_3_max_30_ratio,Close_min_3_min_30_ratio,Close_mean_3_stdev_60_ratio,Close_mean_3_max_60_ratio,Close_mean_3_min_60_ratio,Close_stdev_3_mean_60_ratio,Close_stdev_3_max_60_ratio,Close_stdev_3_min_60_ratio,Close_max_3_mean_60_ratio,Close_max_3_stdev_60_ratio,Close_max_3_min_60_ratio,Close_min_3_mean_60_ratio,Close_min_3_stdev_60_ratio,Close_min_3_max_60_ratio,Close_mean_3_mean_60_ratio,Close_stdev_3_stdev_60_ratio,Close_max_3_max_60_ratio,Close_min_3_min_60_ratio,Close_mean_3_stdev_90_ratio,Close_mean_3_max_90_ratio,Close_mean_3_min_90_ratio,Close_stdev_3_mean_90_ratio,Close_stdev_3_max_90_ratio,Close_stdev_3_min_90_ratio,Close_max_3_mean_90_ratio,Close_max_3_stdev_90_ratio,Close_max_3_min_90_ratio,Close_min_3_mean_90_ratio,Close_min_3_stdev_90_ratio,Close_min_3_max_90_ratio,Close_mean_3_mean_90_ratio,Close_stdev_3_stdev_90_ratio,Close_max_3_max_90_ratio,Close_min_3_min_90_ratio,Close_mean_3_stdev_120_ratio,Close_mean_3_max_120_ratio,Close_mean_3_min_120_ratio,Close_stdev_3_mean_120_ratio,Close_stdev_3_max_120_ratio,Close_stdev_3_min_120_ratio,Close_max_3_mean_120_ratio,Close_max_3_stdev_120_ratio,Close_max_3_min_120_ratio,Close_min_3_mean_120_ratio,Close_min_3_stdev_120_ratio,Close_min_3_max_120_ratio,Close_mean_3_mean_120_ratio,Close_stdev_3_stdev_120_ratio,Close_max_3_max_120_ratio,Close_min_3_min_120_ratio,Close_mean_3_stdev_240_ratio,Close_mean_3_max_240_ratio,Close_mean_3_min_240_ratio,Close_stdev_3_mean_240_ratio,Close_stdev_3_max_240_ratio,Close_stdev_3_min_240_ratio,Close_max_3_mean_240_ratio,Close_max_3_stdev_240_ratio,Close_max_3_min_240_ratio,Close_min_3_mean_240_ratio,Close_min_3_stdev_240_ratio,Close_min_3_max_240_ratio,Close_mean_3_mean_240_ratio,Close_stdev_3_stdev_240_ratio,Close_max_3_max_240_ratio,Close_min_3_min_240_ratio,Close_mean_3_stdev_365_ratio,Close_mean_3_max_365_ratio,Close_mean_3_min_365_ratio,Close_stdev_3_mean_365_ratio,Close_stdev_3_max_365_ratio,Close_stdev_3_min_365_ratio,Close_max_3_mean_365_ratio,Close_max_3_stdev_365_ratio,Close_max_3_min_365_ratio,Close_min_3_mean_365_ratio,Close_min_3_stdev_365_ratio,Close_min_3_max_365_ratio,Close_mean_3_mean_365_ratio,Close_stdev_3_stdev_365_ratio,Close_max_3_max_365_ratio,Close_min_3_min_365_ratio,Close_mean_5_stdev_5_ratio,Close_mean_5_max_5_ratio,Close_mean_5_min_5_ratio,Close_stdev_5_mean_5_ratio,Close_stdev_5_max_5_ratio,Close_stdev_5_min_5_ratio,Close_max_5_mean_5_ratio,Close_max_5_stdev_5_ratio,Close_max_5_min_5_ratio,Close_min_5_mean_5

In [82]:
data4.head()

,Date,Open,High,Low,Close,Volume,Adj Close,month,day,weekday,month_sin,month_cos,weekday_sin,weekday_cos,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
16589,1950-01-03,16.6600,16.6600,16.6600,16.6600,1260000.0000,16.6600,1,3,1,0.5000,0.8660,0.7818,0.6235,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000
16588,1950-01-04,16.8500,16.8500,16.8500,16.8500,1890000.0000,16.8500,1,4,2,0.5000,0.8660,0.9749,-0.2225,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000
16587,1950-01-05,16.9300,16.9300,16.9300,16.9300,2550000.0000,16.9300,1,5,3,0.5000,0.8660,0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000
16586,1950-01-06,16.9800,16.9800,16.9800,16.9800,2010000.0000,16.9800,1,6,4,0.5000,0.8660,-0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000
16585,1950-01-09,17.0800,17.0800,17.0800,17.0800,2520000.0000,17.0800,1,9,0,0.5000,0.8660,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [83]:
# Adding return Features

data4["DailyReturn"] = data4["Close"].pct_change(periods = 1).shift(periods=1) # calculating daily returns and moving values by 1 observation formard
data4['DailyLogReturn'] = np.log1p(data4["DailyReturn"])
data4['CumulativeReturn'] = (1 + data4['DailyReturn']).cumprod()

data6 = gp24feat.VarDescriptiveStatistics(data_in = data4, days = days, var_name = 'DailyReturn', shift = 0).output()
display_html(data6,10)

display_html(data4,10)

(16590, 576)


,DailyReturn_mean_3,DailyReturn_stdev_3,DailyReturn_max_3,DailyReturn_min_3,DailyReturn_mean_5,DailyReturn_stdev_5,DailyReturn_max_5,DailyReturn_min_5,DailyReturn_mean_30,DailyReturn_stdev_30,DailyReturn_max_30,DailyReturn_min_30,DailyReturn_mean_60,DailyReturn_stdev_60,DailyReturn_max_60,DailyReturn_min_60,DailyReturn_mean_90,DailyReturn_stdev_90,DailyReturn_max_90,DailyReturn_min_90,DailyReturn_mean_120,DailyReturn_stdev_120,DailyReturn_max_120,DailyReturn_min_120,DailyReturn_mean_240,DailyReturn_stdev_240,DailyReturn_max_240,DailyReturn_min_240,DailyReturn_mean_365,DailyReturn_stdev_365,DailyReturn_max_365,DailyReturn_min_365,DailyReturn_mean_3_stdev_3_ratio,DailyReturn_mean_3_max_3_ratio,DailyReturn_mean_3_min_3_ratio,DailyReturn_stdev_3_mean_3_ratio,DailyReturn_stdev_3_max_3_ratio,DailyReturn_stdev_3_min_3_ratio,DailyReturn_max_3_mean_3_ratio,DailyReturn_max_3_stdev_3_ratio,DailyReturn_max_3_min_3_ratio,DailyReturn_min_3_mean_3_ratio,DailyReturn_min_3_stdev_3_ratio,DailyReturn_min_3_max_3_ratio,DailyReturn_mean_3_stdev_5_ratio,DailyReturn_mean_3_max_5_ratio,DailyReturn_mean_3_min_5_ratio,DailyReturn_stdev_3_mean_5_ratio,DailyReturn_stdev_3_max_5_ratio,DailyReturn_stdev_3_min_5_ratio,DailyReturn_max_3_mean_5_ratio,DailyReturn_max_3_stdev_5_ratio,DailyReturn_max_3_min_5_ratio,DailyReturn_min_3_mean_5_ratio,DailyReturn_min_3_stdev_5_ratio,DailyReturn_min_3_max_5_ratio,DailyReturn_mean_3_mean_5_ratio,DailyReturn_stdev_3_stdev_5_ratio,DailyReturn_max_3_max_5_ratio,DailyReturn_min_3_min_5_ratio,DailyReturn_mean_3_stdev_30_ratio,DailyReturn_mean_3_max_30_ratio,DailyReturn_mean_3_min_30_ratio,DailyReturn_stdev_3_mean_30_ratio,DailyReturn_stdev_3_max_30_ratio,DailyReturn_stdev_3_min_30_ratio,DailyReturn_max_3_mean_30_ratio,DailyReturn_max_3_stdev_30_ratio,DailyReturn_max_3_min_30_ratio,DailyReturn_min_3_mean_30_ratio,DailyReturn_min_3_stdev_30_ratio,DailyReturn_min_3_max_30_ratio,DailyReturn_mean_3_mean_30_ratio,DailyReturn_stdev_3_stdev_30_ratio,DailyReturn_max_3_max_30_ratio,DailyReturn_min_3_min_30_ratio,DailyReturn_mean_3_stdev_60_ratio,DailyReturn_mean_3_max_60_ratio,DailyReturn_mean_3_min_60_ratio,DailyReturn_stdev_3_mean_60_ratio,DailyReturn_stdev_3_max_60_ratio,DailyReturn_stdev_3_min_60_ratio,DailyReturn_max_3_mean_60_ratio,DailyReturn_max_3_stdev_60_ratio,DailyReturn_max_3_min_60_ratio,DailyReturn_min_3_mean_60_ratio,DailyReturn_min_3_stdev_60_ratio,DailyReturn_min_3_max_60_ratio,DailyReturn_mean_3_mean_60_ratio,DailyReturn_stdev_3_stdev_60_ratio,DailyReturn_max_3_max_60_ratio,DailyReturn_min_3_min_60_ratio,DailyReturn_mean_3_stdev_90_ratio,DailyReturn_mean_3_max_90_ratio,DailyReturn_mean_3_min_90_ratio,DailyReturn_stdev_3_mean_90_ratio,DailyReturn_stdev_3_max_90_ratio,DailyReturn_stdev_3_min_90_ratio,DailyReturn_max_3_mean_90_ratio,DailyReturn_max_3_stdev_90_ratio,DailyReturn_max_3_min_90_ratio,DailyReturn_min_3_mean_90_ratio,DailyReturn_min_3_stdev_90_ratio,DailyReturn_min_3_max_90_ratio,DailyReturn_mean_3_mean_90_ratio,DailyReturn_stdev_3_stdev_90_ratio,DailyReturn_max_3_max_90_ratio,DailyReturn_min_3_min_90_ratio,DailyReturn_mean_3_stdev_120_ratio,DailyReturn_mean_3_max_120_ratio,DailyReturn_mean_3_min_120_ratio,DailyReturn_stdev_3_mean_120_ratio,DailyReturn_stdev_3_max_120_ratio,DailyReturn_stdev_3_min_120_ratio,DailyReturn_max_3_mean_120_ratio,DailyReturn_max_3_stdev_120_ratio,DailyReturn_max_3_min_120_ratio,DailyReturn_min_3_mean_120_ratio,DailyReturn_min_3_stdev_120_ratio,DailyReturn_min_3_max_120_ratio,DailyReturn_mean_3_mean_120_ratio,DailyReturn_stdev_3_stdev_120_ratio,DailyReturn_max_3_max_120_ratio,DailyReturn_min_3_min_120_ratio,DailyReturn_mean_3_stdev_240_ratio,DailyReturn_mean_3_max_240_ratio,DailyReturn_mean_3_min_240_ratio,DailyReturn_stdev_3_mean_240_ratio,DailyReturn_stdev_3_max_240_ratio,DailyReturn_stdev_3_min_240_ratio,DailyReturn_max_3_mean_240_ratio,DailyReturn_max_3_stdev_240_ratio,DailyReturn_max_3_min_240_ratio,DailyReturn_min_3_mean_240_ratio,DailyReturn_min_3_stdev_240_ratio,DailyReturn_min

(16590, 36)


,Date,Open,High,Low,Close,Volume,Adj Close,month,day,weekday,month_sin,month_cos,weekday_sin,weekday_cos,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,DailyReturn,DailyLogReturn,CumulativeReturn
16589,1950-01-03,16.6600,16.6600,16.6600,16.6600,1260000.0000,16.6600,1,3,1,0.5000,0.8660,0.7818,0.6235,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,NaN,NaN,NaN
16588,1950-01-04,16.8500,16.8500,16.8500,16.8500,1890000.0000,16.8500,1,4,2,0.5000,0.8660,0.9749,-0.2225,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,NaN,NaN,NaN
16587,1950-01-05,16.9300,16.9300,16.9300,16.9300,2550000.0000,16.9300,1,5,3,0.5000,0.8660,0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0114,0.0113,1.0114
16586,1950-01-06,16.9800,16.9800,16.9800,16.9800,2010000.0000,16.9800,1,6,4,0.5000,0.8660,-0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0047,0.0047,1.0162
16585,1950-01-09,17.0800,17.0800,17.0800,17.0800,2520000.0000,17.0800,1,9,0,0.5000,0.8660,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0030,0.0029,1.0192
16584,1950-01-10,17.0300,17.0300,17.0300,17.0300,2160000.0000,17.0300,1,10,1,0.5000,0.8660,0.7818,0.6235,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0059,0.0059,1.0252
16583,1950-01-11,17.0900,17.0900,17.0900,17.0900,2630000.0000,17.0900,1,11,2,0.5000,0.8660,0.9749,-0.2225,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,-0.0029,-0.0029,1.0222
16582,1950-01-12,16.7600,16.7600,16.7600,16.7600,2970000.0000,16.7600,1,12,3,0.5000,0.8660,0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0035,0.0035,1.0258
16581,1950-01-13,16.6700,16.6700,16.6700,16.6700,3330000.0000,16.6700,1,13,4,0.5000,0.8660,-0.4339,-0.9010,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,-0.0193,-0.0195,1.0060
16580,1950-01-16,16.7200,16.7200,16.7200,16.7200,1460000.0000,16.7200,1,16,0,0.5000,0.8660,0.0000,1.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0054,-0.0054,1.0006


In [84]:
data7 = pd.concat([data4, data5, data6], axis = 1)
display_html(data7, 10)

(16590, 1188)


In [85]:
class technicalAnalysisFE:
    def __init__(self, data_in, low = 'Low', high = 'High',  close = 'Close', volume = 'Volume'):
        self.data_in = data_in
        self.data_out = pd.DataFrame()
        self.open = open
        self.low = low
        self.high = high
        self.close = close
        self.volume = volume
        self._validate_inputs()
        self._CalculateFeatures()

    def _validate_inputs(self):
        if self.data_in is None or self.data_in.empty:
            raise ValueError("Data cannot be None or empty.")
        
        var_list = []
        self.var_list_ind = {}
        var_list.append(self.low)
        var_list.append(self.high)
        var_list.append(self.close)
        var_list.append(self.volume)

        for variable in var_list:
            count = 0
            if variable not in self.data_in.columns:
                count += 1
                self.var_list_ind[variable] = 0
                raise Warning(f"Variable {variable} is not present in the data.")
            else:
                self.var_list_ind[variable] = 1
        if count == 4:
            raise ValueError("Pointed variables not present in the data")
        
    def _CalculateFeatures(self):
        print(self.var_list_ind[self.close])
        # if close variable is present then calculate
        if self.var_list_ind[self.close] == 1:
            # https://www.tradingview.com/ideas/kama/
            self.data_out['KAMA_30'] = ta.momentum.kama(self.data_in['Close'])

            # https://school.stockcharts.com/doku.php?id=technical_indicators:price_oscillators_ppo
            self.data_out['PercentagePriceOscillator_30'] = ta.momentum.ppo(self.data_in[self.close])
            self.data_out['PercentagePriceOscillator_Hist_30'] = ta.momentum.ppo_hist(self.data_in[self.close])
            self.data_out['PercentagePriceOscillator_Signal_30'] = ta.momentum.ppo_signal(self.data_in[self.close])

            # https://school.stockcharts.com/doku.php?id=technical_indicators:percentage_volume_oscillator_pvo
            self.data_out['PercentageVolumeOscillator_26'] = ta.momentum.pvo(self.data_in[self.close])
            self.data_out['PercentageVolumeOscillator_Hist_26'] = ta.momentum.pvo_hist(self.data_in[self.close])
            self.data_out['PercentageVolumeOscillator_Signal_26'] = ta.momentum.pvo_signal(self.data_in[self.close])

            # https://school.stockcharts.com/doku.php?id=technical_indicators:rate_of_change_roc_and_momentum
            self.data_out['RateOfChange_12'] = ta.momentum.roc(self.data_in[self.close])

            # https://www.investopedia.com/terms/r/rsi.asp
            self.data_out['RelativeStrengthIndex_14'] = ta.momentum.rsi(self.data_in[self.close], window=14)


            # https://school.stockcharts.com/doku.php?id=technical_indicators:stochrsi
            self.data_out['StochasticRelativeStrengthIndex_14'] = ta.momentum.stochrsi(self.data_in[self.close])
            self.data_out['StochasticRelativeStrengthIndex_pd_14'] = ta.momentum.stochrsi_d(self.data_in[self.close])
            self.data_out['StochasticRelativeStrengthIndex_pk_14'] = ta.momentum.stochrsi_k(self.data_in[self.close])

            # Boilinger Bands
            # https://school.stockcharts.com/doku.php?id=technical_indicators:bollinger_bands

            # Bollinger high band
            self.data_out['BollingerHighBand_20'] = ta.volatility.bollinger_hband(close = self.data_in[self.close])
            # Bollinger high band indicator
            self.data_out['BollingerHighBandIndicator_20'] = ta.volatility.bollinger_hband_indicator(close = self.data_in[self.close])
            # Bollinger low band
            self.data_out['BollingerLowBand_20'] = ta.volatility.bollinger_lband(close = self.data_in[self.close])
            # Bolligner low band indicator
            self.data_out['BollingerLowBandIndicator_20'] = ta.volatility.bollinger_lband_indicator(close = self.data_in[self.close])
            # Bollinger Channel Middle Band
            self.data_out['BollingerChannelMiddleBand_20'] = ta.volatility.bollinger_mavg(close = self.data_in[self.close])
            # Bollinger Channel Percentage Band
            self.data_out['BollingerChannelPercentageBand_20'] = ta.volatility.bollinger_pband(close = self.data_in[self.close])
            # Bollinger Channel Band Width
            self.data_out['BollingerChannelBandWith_20'] = ta.volatility.bollinger_wband(close = self.data_in[self.close])

            # Volatility Ulcer Index
            # https://school.stockcharts.com/doku.php?id=technical_indicators:ulcer_index
            self.data_out['VolatilityUlcerIndex_14'] = ta.volatility.ulcer_index(close = self.data_in[self.close])

            # WMA - Weighted Moving Average (WMA) Indicator
            # https://school.stockcharts.com/doku.php?id=technical_indicators:trix
            self.data_out['WeightedMovingAverage_9'] = ta.trend.WMAIndicator(close = self.data_in[self.close]).wma()

            # Aroon Indicator
            # https://www.investopedia.com/terms/a/aroon.asp
            self.data_out['AroonUpChannel_25'] = ta.trend.aroon_up(close = self.data_in[self.close])
            self.data_out['AroonDownChannel_25'] = ta.trend.aroon_down(close = self.data_in[self.close])
            self.data_out['AroonIndicator_25'] = ta.trend.AroonIndicator(close = self.data_in[self.close]).aroon_indicator()

            # Detrended Price Oscillator (DPO)
            # https://school.stockcharts.com/doku.php?id=technical_indicators:commodity_channel_index_cci
            self.data_out['DetrendedPriceOscillator_20'] = ta.trend.dpo(close = self.data_in[self.close])

            # EMA - Exponential Moving Average
            # https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages
            self.data_out['ExponentialMovingAverage_14'] = ta.trend.ema_indicator(close = self.data_in[self.close])

            # KST Oscillator (KST Signal)
            # https://school.stockcharts.com/doku.php?id=technical_indicators:know_sure_thing_kst
            self.data_out['KnowSureThingOscillator_30'] = ta.trend.kst(close = self.data_in[self.close])
            self.data_out['KnowSureThingDifference_30'] = ta.trend.KSTIndicator(close = self.data_in[self.close]).kst_diff()
            self.data_out['KnowSureThingSignal_30'] = ta.trend.kst_sig(close = self.data_in[self.close])

            # Moving Average Convergence Divergence (MACD)
            # https://school.stockcharts.com/doku.php?id=technical_indicators:moving_average_convergence_divergence_macd

            self.data_out['MovingAverageConvergenceDivergence_26'] = ta.trend.macd(close = self.data_in[self.close])
            self.data_out['MovingAverageConvergenceDivergenceHistogram_26'] = ta.trend.macd_diff(close = self.data_in[self.close])
            self.data_out['MovingAverageConvergenceDivergenceSignal_26'] = ta.trend.macd_signal(close = self.data_in[self.close])

            # SMA - Simple Moving Average
            # https://school.stockcharts.com/doku.php?id=technical_indicators:moving_averages
            self.data_out['SimpleMovingAverage'] = ta.trend.sma_indicator(close = self.data_in[self.close])

            # Schaff Trend Cycle (STC) Indicator
            # https://www.investopedia.com/articles/forex/10/schaff-trend-cycle-indicator.asp
            self.data_out['SchaffTrendCycleIndicator_50'] = ta.trend.stc(close = self.data_in[self.close])

            # Trix (TRIX) Indicator
            # https://school.stockcharts.com/doku.php?id=technical_indicators:trix
            self.data_out['TRIXIndicator_15'] = ta.trend.trix(close = self.data_in[self.close])

            # True Strength Index
            # https://school.stockcharts.com/doku.php?id=technical_indicators:true_strength_index
            self.data_out['TrueStrengthIndex_14'] = ta.momentum.tsi(close = self.data_in[self.close])
            
            # If both close and volume variables are present then calculate
            if self.var_list_ind[self.volume] == 1:

                # Force Index
                # https://school.stockcharts.com/doku.php?id=technical_indicators:force_index
                self.data_out['ForceIndex_13'] = ta.volume.force_index(close = self.data_in[self.close], volume = self.data_in[self.volume])

                # Negative Volume Index
                # stockcharts.com/school/doku.php?id=chart_school:technical_indicators:negative_volume_inde
                self.data_out['NegativeVolumeIndex_2'] = ta.volume.negative_volume_index(close = self.data_in[self.close], volume = self.data_in[self.volume])

                # On-balance Volume
                # https://en.wikipedia.org/wiki/On-balance_volume
                self.data_out['OnBalanceVolume_2'] = ta.volume.on_balance_volume(close = self.data_in[self.close], volume = self.data_in[self.volume])

                # Volume Price Trend
                # https://en.wikipedia.org/wiki/Volume%E2%80%93price_trend
                self.data_out['VolumePriceTrend_2'] = ta.volume.volume_price_trend(close = self.data_in[self.close], volume = self.data_in[self.volume])

                if self.var_list_ind[self.low] == 1 and self.var_list_ind[self.high] == 1:

                    # Accumulation Distribution Line Index
                    # https://school.stockcharts.com/doku.php?id=technical_indicators:accumulation_distribution_line
                    self.data_out['AccumulationDistributionLineIndex_1'] = ta.volume.acc_dist_index(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close], volume = self.data_in[self.volume])

                    # Chaikin Money Flow
                    # https://school.stockcharts.com/doku.php?id=technical_indicators:chaikin_money_flow_cmf
                    self.data_out['ChaikinMoneyFlow_20'] = ta.volume.chaikin_money_flow(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close], volume = self.data_in[self.volume])

                    # Ease of Movement
                    # https://en.wikipedia.org/wiki/Ease_of_movement
                    self.data_out['EaseOfMovement_14'] = ta.volume.ease_of_movement(high = self.data_in[self.high], low = self.data_in[self.low], volume = self.data_in[self.volume])
                    self.data_out['SimpleMovingAverageEaseOfMovement_14'] = ta.volume.sma_ease_of_movement(high = self.data_in[self.high], low = self.data_in[self.low], volume = self.data_in[self.volume])

                    # Money Flow Index
                    # http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:money_flow_index_mfi
                    self.data_out['MoneyFlowIndex_14'] = ta.volume.money_flow_index(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close], volume = self.data_in[self.volume])

                    # Volume Weighted Average Price
                    # https://school.stockcharts.com/doku.php?id=technical_indicators:vwap_intraday
                    self.data_out['VolumeWeightedAveragePrice_14'] = ta.volume.volume_weighted_average_price(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close], volume = self.data_in[self.volume])
            if self.var_list_ind[self.low] == 1 and self.var_list_ind[self.high] == 1:

                # Stochastic Oscillator
                # https://school.stockcharts.com/doku.php?id=technical_indicators:stochastic_oscillator_fast_slow_and_full
                self.data_out['StochasticOscillator_14'] = ta.momentum.stoch(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                self.data_out['StochasticOscillatorSignal_14'] = ta.momentum.stoch(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])

                # Ultimate Oscillator
                # https://school.stockcharts.com/doku.php?id=technical_indicators:ultimate_oscillator
                self.data_out['UltimateOscillator_28'] = ta.momentum.ultimate_oscillator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])

                # Williams %R
                # https://school.stockcharts.com/doku.php?id=technical_indicators:williams_r
                self.data_out['WilliamsPercentageR_14'] = ta.momentum.williams_r(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])

                # Average True Range (14 days window)
                # https://school.stockcharts.com/doku.php?id=technical_indicators:average_true_range_atr
                self.data_out['AverateTrueRange_14'] = ta.volatility.average_true_range(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])

                # Donchian Channel
                # https://www.investopedia.com/terms/d/donchianchannels.asp
                # Donchian Channel High Band 
                self.data_out['DonchianChannelHighBand_20'] = ta.volatility.donchian_channel_hband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Donchian Channel Low Band 
                self.data_out['DonchianChannelLowBand_20'] = ta.volatility.donchian_channel_lband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Donchian Channel Middle Band 
                self.data_out['DonchianChannelMiddleBand_20'] = ta.volatility.donchian_channel_mband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Donchian Channel Percentage Band 
                self.data_out['DonchianChannelPercentageBand_20'] = ta.volatility.donchian_channel_pband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Donchian Channel Band width 
                self.data_out['DonchianChannelBandWidth_20'] = ta.volatility.donchian_channel_wband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])

                # Keltner Channel
                # https://school.stockcharts.com/doku.php?id=technical_indicators:keltner_channels
                # Keltner Channel High Band 
                self.data_out['KeltnerChannelHighBand_20'] = ta.volatility.keltner_channel_hband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Keltner Channel High Band Indicator
                self.data_out['KeltnerChannelHighBandIndicator_20'] = ta.volatility.keltner_channel_hband_indicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Keltner Channel Low Band 
                self.data_out['KeltnerChannelLowBand_20'] = ta.volatility.keltner_channel_lband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Keltner Channel Low Band Indicator
                self.data_out['KeltnerChannelLowBandIndicator_20'] = ta.volatility.keltner_channel_lband_indicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Keltner Channel Middle Band 
                self.data_out['KeltnerChannelMiddleBand_20'] = ta.volatility.keltner_channel_mband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Keltner Channel Percentage Band 
                self.data_out['KeltnerChannelPercentageBand_20'] = ta.volatility.keltner_channel_pband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                # Keltner Channel Band width 
                self.data_out['KeltnerChannelBandWidth_20'] = ta.volatility.keltner_channel_wband(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])

                # Average Directional Index (ADX)
                # https://school.stockcharts.com/doku.php?id=technical_indicators:average_directional_index_adx 
                # Average Directional Index (ADX)
                self.data_out['AverageDirectionalIndex_14'] = ta.trend.ADXIndicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close]).adx()
                # Minus Directional Indicator (-DI)
                self.data_out['AverageDirectionalNegativeIndex_14'] = ta.trend.ADXIndicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close]).adx_neg()
                # Plus Directional Indicator (+DI)
                self.data_out['AverageDirectionalPostiiveIndex_14'] = ta.trend.ADXIndicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close]).adx_pos()

                # Commodity Channel Index (CCI)
                # https://school.stockcharts.com/doku.php?id=technical_indicators:commodity_channel_index_cci
                self.data_out['CommodityChannelIndex_20'] = ta.trend.CCIIndicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close]).cci()

                # Parabolic Stop and Reverse (Parabolic SAR)
                # https://school.stockcharts.com/doku.php?id=technical_indicators:parabolic_sar

                self.data_out['ParabolicStopAndReverse_25'] = ta.trend.PSARIndicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close]).psar()
                self.data_out['ParabolicStopAndReverseDown_25'] = ta.trend.psar_down(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                self.data_out['ParabolicStopAndReverseDownIndicator_25'] = ta.trend.psar_down_indicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                self.data_out['ParabolicStopAndReverseUp_25'] = ta.trend.psar_up(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                self.data_out['ParabolicStopAndReverseUpIndicator_25'] = ta.trend.psar_up_indicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])

                # Vortex Indicator (VI)
                # https://school.stockcharts.com/doku.php?id=technical_indicators:trix

                self.data_out['VortexIndicatorDifference_14'] = ta.trend.VortexIndicator(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close]).vortex_indicator_diff()
                self.data_out['VortexIndicatorNegative_14'] = ta.trend.vortex_indicator_neg(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
                self.data_out['VortexIndicatorPositive_14'] = ta.trend.vortex_indicator_pos(high = self.data_in[self.high], low = self.data_in[self.low], close = self.data_in[self.close])
        if self.var_list_ind[self.low] == 1 and self.var_list_ind[self.high] == 1:
            # Awesome Oscillator
            # https://www.ifcm.co.uk/ntx-indicators/awesome-oscillator
            self.data_out['AwesomeOscillatorIndicator_34'] = ta.momentum.awesome_oscillator(high = self.data_in[self.high], low = self.data_in[self.low])  # n = 34

            # IchimokuIndicator
            # https://school.stockcharts.com/doku.php?id=technical_indicators:ichimoku_cloud
            # Senoku span a
            self.data_out['IchimokuSenokuSpanA_52'] = ta.trend.ichimoku_a(high = self.data_in[self.high], low = self.data_in[self.low])
            # Senoku span b
            self.data_out['IchimokuSenokuSpanB_52'] = ta.trend.ichimoku_b(high = self.data_in[self.high], low = self.data_in[self.low])
            # Ichimoku baseline
            self.data_out['IchimokuBaseline_52'] = ta.trend.ichimoku_base_line(high = self.data_in[self.high], low = self.data_in[self.low])
            # Ichimoku conversion line
            self.data_out['IchimokuConversionLine_52'] = ta.trend.ichimoku_conversion_line(high = self.data_in[self.high], low = self.data_in[self.low])


            # Mass Index (MI)
            # https://school.stockcharts.com/doku.php?id=technical_indicators:mass_index
            self.data_out['MassIndex_25'] = ta.trend.mass_index(high = self.data_in[self.high], low = self.data_in[self.low])


        self.data_out.shift() # moving 1 observation forward

        return self.data_out

In [86]:
#close	low 	high	volume
#1			
#1	    1	    1	
#1	    1	    1	    1
#1			            1
#	    1	    1	
#	    1	    1	    1


In [87]:
dupczok = technicalAnalysisFE(data_in = data7)
print(dupczok.var_list_ind)
display_html(dupczok.data_out, 30)

1


c:\ProgramData\Anaconda3\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
c:\ProgramData\Anaconda3\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


{'Low': 1, 'High': 1, 'Close': 1, 'Volume': 1}
(16590, 81)


,KAMA_30,PercentagePriceOscillator_30,PercentagePriceOscillator_Hist_30,PercentagePriceOscillator_Signal_30,PercentageVolumeOscillator_26,PercentageVolumeOscillator_Hist_26,PercentageVolumeOscillator_Signal_26,RateOfChange_12,RelativeStrengthIndex_14,StochasticRelativeStrengthIndex_14,StochasticRelativeStrengthIndex_pd_14,StochasticRelativeStrengthIndex_pk_14,BollingerHighBand_20,BollingerHighBandIndicator_20,BollingerLowBand_20,BollingerLowBandIndicator_20,BollingerChannelMiddleBand_20,BollingerChannelPercentageBand_20,BollingerChannelBandWith_20,VolatilityUlcerIndex_14,WeightedMovingAverage_9,AroonUpChannel_25,AroonDownChannel_25,AroonIndicator_25,DetrendedPriceOscillator_20,ExponentialMovingAverage_14,KnowSureThingOscillator_30,KnowSureThingDifference_30,KnowSureThingSignal_30,MovingAverageConvergenceDivergence_26,MovingAverageConvergenceDivergenceHistogram_26,MovingAverageConvergenceDivergenceSignal_26,SimpleMovingAverage,SchaffTrendCycleIndicator_50,TRIXIndicator_15,TrueStrengthIndex_14,ForceIndex_13,NegativeVolumeIndex_2,OnBalanceVolume_2,VolumePriceTrend_2,AccumulationDistributionLineIndex_1,ChaikinMoneyFlow_20,EaseOfMovement_14,SimpleMovingAverageEaseOfMovement_14,MoneyFlowIndex_14,VolumeWeightedAveragePrice_14,StochasticOscillator_14,StochasticOscillatorSignal_14,UltimateOscillator_28,WilliamsPercentageR_14,AverateTrueRange_14,DonchianChannelHighBand_20,DonchianChannelLowBand_20,DonchianChannelMiddleBand_20,DonchianChannelPercentageBand_20,DonchianChannelBandWidth_20,KeltnerChannelHighBand_20,KeltnerChannelHighBandIndicator_20,KeltnerChannelLowBand_20,KeltnerChannelLowBandIndicator_20,KeltnerChannelMiddleBand_20,KeltnerChannelPercentageBand_20,KeltnerChannelBandWidth_20,AverageDirectionalIndex_14,AverageDirectionalNegativeIndex_14,AverageDirectionalPostiiveIndex_14,CommodityChannelIndex_20,ParabolicStopAndReverse_25,ParabolicStopAndReverseDown_25,ParabolicStopAndReverseDownIndicator_25,ParabolicStopAndReverseUp_25,ParabolicStopAndReverseUpIndicator_25,VortexIndicatorDifference_14,VortexIndicatorNegative_14,VortexIndicatorPositive_14,AwesomeOscillatorIndicator_34,IchimokuSenokuSpanA_52,IchimokuSenokuSpanB_52,IchimokuBaseline_52,IchimokuConversionLine_52,MassIndex_25
16589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0000,1260000.0000,-1072837.3152,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,16.6600,0.0000,16.6600,0.0000,NaN,NaN,NaN,0.0000,0.0000,0.0000,NaN,16.6600,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,16.6600,NaN,NaN,NaN
16588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0000,3150000.0000,-1194956.8210,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,16.7550,1.0000,16.7550,0.0000,NaN,inf,NaN,0.0000,0.0000,0.0000,NaN,2104.2700,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,16.7550,NaN,NaN,NaN
16587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.0000,5700000.0000,33661.4468,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,16.8133,1.0000,16.8133,0.0000,NaN,inf,NaN,0.0000,0.0000,0.0000,NaN,16.6600,NaN,0.0000,16.6600,1.0000,NaN,NaN,NaN,NaN,NaN,16.7950,NaN,NaN,NaN
16586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1002.9533,7710000.0000,18043.0328,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,16.8550,1.0000,16.8550,0.0000,NaN,inf,NaN,0.0000,0.0000,0.0000,NaN,16.6708,NaN,0.0000,16.6708,0.0000,NaN,NaN,NaN,NaN,NaN,16.8200,NaN,NaN,NaN
16585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [88]:
data7.head()

,Date,Open,High,Low,Close,Volume,Adj Close,month,day,weekday,month_sin,month_cos,weekday_sin,weekday_cos,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,DailyReturn,DailyLogReturn,CumulativeReturn,Close_mean_3,Close_stdev_3,Close_max_3,Close_min_3,Close_mean_5,Close_stdev_5,Close_max_5,Close_min_5,Close_mean_30,Close_stdev_30,Close_max_30,Close_min_30,Close_mean_60,Close_stdev_60,Close_max_60,Close_min_60,Close_mean_90,Close_stdev_90,Close_max_90,Close_min_90,Close_mean_120,Close_stdev_120,Close_max_120,Close_min_120,Close_mean_240,Close_stdev_240,Close_max_240,Close_min_240,Close_mean_365,Close_stdev_365,Close_max_365,Close_min_365,Close_mean_3_stdev_3_ratio,Close_mean_3_max_3_ratio,Close_mean_3_min_3_ratio,Close_stdev_3_mean_3_ratio,Close_stdev_3_max_3_ratio,Close_stdev_3_min_3_ratio,Close_max_3_mean_3_ratio,Close_max_3_stdev_3_ratio,Close_max_3_min_3_ratio,Close_min_3_mean_3_ratio,Close_min_3_stdev_3_ratio,Close_min_3_max_3_ratio,Close_mean_3_stdev_5_ratio,Close_mean_3_max_5_ratio,Close_mean_3_min_5_ratio,Close_stdev_3_mean_5_ratio,Close_stdev_3_max_5_ratio,Close_stdev_3_min_5_ratio,Close_max_3_mean_5_ratio,Close_max_3_stdev_5_ratio,Close_max_3_min_5_ratio,Close_min_3_mean_5_ratio,Close_min_3_stdev_5_ratio,Close_min_3_max_5_ratio,Close_mean_3_mean_5_ratio,Close_stdev_3_stdev_5_ratio,Close_max_3_max_5_ratio,Close_min_3_min_5_ratio,Close_mean_3_stdev_30_ratio,Close_mean_3_max_30_ratio,Close_mean_3_min_30_ratio,Close_stdev_3_mean_30_ratio,Close_stdev_3_max_30_ratio,Close_stdev_3_min_30_ratio,Close_max_3_mean_30_ratio,Close_max_3_stdev_30_ratio,Close_max_3_min_30_ratio,Close_min_3_mean_30_ratio,Close_min_3_stdev_30_ratio,Close_min_3_max_30_ratio,Close_mean_3_mean_30_ratio,Close_stdev_3_stdev_30_ratio,Close_max_3_max_30_ratio,Close_min_3_min_30_ratio,Close_mean_3_stdev_60_ratio,Close_mean_3_max_60_ratio,Close_mean_3_min_60_ratio,Close_stdev_3_mean_60_ratio,Close_stdev_3_max_60_ratio,Close_stdev_3_min_60_ratio,Close_max_3_mean_60_ratio,Close_max_3_stdev_60_ratio,Close_max_3_min_60_ratio,Close_min_3_mean_60_ratio,Close_min_3_stdev_60_ratio,Close_min_3_max_60_ratio,Close_mean_3_mean_60_ratio,Close_stdev_3_stdev_60_ratio,Close_max_3_max_60_ratio,Close_min_3_min_60_ratio,Close_mean_3_stdev_90_ratio,Close_mean_3_max_90_ratio,Close_mean_3_min_90_ratio,Close_stdev_3_mean_90_ratio,Close_stdev_3_max_90_ratio,Close_stdev_3_min_90_ratio,Close_max_3_mean_90_ratio,Close_max_3_stdev_90_ratio,Close_max_3_min_90_ratio,Close_min_3_mean_90_ratio,Close_min_3_stdev_90_ratio,Close_min_3_max_90_ratio,Close_mean_3_mean_90_ratio,Close_stdev_3_stdev_90_ratio,Close_max_3_max_90_ratio,Close_min_3_min_90_ratio,Close_mean_3_stdev_120_ratio,Close_mean_3_max_120_ratio,Close_mean_3_min_120_ratio,Close_stdev_3_mean_120_ratio,Close_stdev_3_max_120_ratio,Close_stdev_3_min_120_ratio,...,DailyReturn_min_60_stdev_365_ratio,DailyReturn_min_60_max_365_ratio,DailyReturn_mean_60_mean_365_ratio,DailyReturn_stdev_60_stdev_365_ratio,DailyReturn_max_60_max_365_ratio,DailyReturn_min_60_min_365_ratio,DailyReturn_mean_90_stdev_90_ratio,DailyReturn_mean_90_max_90_ratio,DailyReturn_mean_90_min_90_ratio,DailyReturn_stdev_90_mean_90_ratio,DailyReturn_stdev_90_max_90_ratio,DailyReturn_stdev_90_min_90_ratio,DailyReturn_max_90_mean_90_ratio,DailyReturn_max_90_stdev_90_ratio,DailyReturn_max_90_min_90_ratio,DailyReturn_min_90_mean_90_ratio,DailyReturn_min_90_stdev_90_ratio,DailyReturn_min_90_max_90_ratio,DailyReturn_mean_90_stdev_120_ratio,DailyReturn_mean_90_max_120_ratio,DailyReturn_mean_90_min_120_ratio,DailyReturn_stdev_90_mean_120_ratio,DailyReturn_stdev_90_max_120_ratio,DailyReturn_stdev_90_min_120_ratio,DailyReturn_max_90_mean_120_ratio,DailyReturn_max_90_stdev_120_ratio,DailyReturn_max_90_min_120_ratio,DailyReturn_min_90_mean_120_ratio,DailyReturn_min_90_stdev_120_ratio,DailyReturn_min_90_max_120_ratio,DailyReturn_mean_90_mean_120_ratio,DailyReturn_stdev_90_st

In [89]:
kuciapkacz

NameError: name 'kuciapkacz' is not defined

In [ ]:
# Technical Analysis Indicators
# te funkcje zwracaja obiekty, a nie liczby - do sprawdzenia
self.data_out = pd.DataFrame()

# Momentum Indicators




self.data_out.info()
display_html(self.data_out, 35)


c:\ProgramData\Anaconda3\lib\site-packages\ta\trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
c:\ProgramData\Anaconda3\lib\site-packages\ta\trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


<class 'pandas.core.frame.DataFrame'>
Index: 16590 entries, 16589 to 0
Data columns (total 84 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AOI_34            16557 non-null  float64
 1   KAMA_30           16581 non-null  float64
 2   PPO_30            16565 non-null  float64
 3   PPO_Hist_30       16557 non-null  float64
 4   PPO_Signal_30     16557 non-null  float64
 5   PVO_26            16565 non-null  float64
 6   PVO_Hist_26       16557 non-null  float64
 7   PVO_Signal_26     16557 non-null  float64
 8   ROC_12            16578 non-null  float64
 9   RSI_14            16577 non-null  float64
 10  SRSI_14           16564 non-null  float64
 11  SRSId_14          16560 non-null  float64
 12  SRSIk_14          16562 non-null  float64
 13  SO_14             16577 non-null  float64
 14  SO_Signal_14      16577 non-null  float64
 15  TSII_14           16553 non-null  float64
 16  UO_28             16562 non-null  float64
 17

,AOI_34,KAMA_30,PPO_30,PPO_Hist_30,PPO_Signal_30,PVO_26,PVO_Hist_26,PVO_Signal_26,ROC_12,RSI_14,SRSI_14,SRSId_14,SRSIk_14,SO_14,SO_Signal_14,TSII_14,UO_28,WI_14,ADII,CMFI,EoM_14,SMAEoM_14,FI_13,MFI_14,NVI_2,OBV_2,VPT_2,VWAP_14,ATR_14,BhB_20,BhBI_20,BlB_20,BlBI_20,BcmB_20,BcpB_20,BcBw_20,DChB_20,DClB_20,DCmB_20,DCpB_20,DCBw_20,KChB_20,KChBI_20,KClB_20,KClBI_20,KCmB_20,KCpB_20,KCBw_20,UI_14,ADX_14,ADXnI_14,ADXpI_14,ACu_25,ACd_25,ACI_25,CCI_20,DPO_20,EMA_14,ISa_52,ISb_52,Ibl_52,Icl_52,KST_30,KSTdiff_30,KSTsig_30,MACD_26,MACDhist_26,MACDsig_26,MI_25,PSAR_25,PSARd_25,PSARdI_25,PSARu_25,PSARuI_25,SMA,STCI_50,TRIX_15,VIdiff_14,VIn_14,VIp_14,WMA_9,CumulativeReturn,DailyLogReturn,DailyReturn
16589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,1000.0000,1260000.0000,-1072837.3152,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6600,0.0000,16.6600,0.0000,NaN,NaN,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6600,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,-96.5485
16588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,1000.0000,3150000.0000,-1194956.8210,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7550,1.0000,16.7550,0.0000,NaN,inf,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2104.2700,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1405,1.1340,1.1405
16587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,1000.0000,5700000.0000,33661.4468,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.8133,1.0000,16.8133,0.0000,NaN,inf,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6600,NaN,0.0000,16.6600,1.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6206,0.4737,0.4748
16586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,1002.9533,7710000.0000,18043.0328,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.8550,1.0000,16.8550,0.0000,NaN,inf,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.8200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6708,NaN,0.0000,16.6708,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.9208,0.2949,0.2953
16585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,1002.9533,10230000.0000,20777.1973,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.9000,1.0000,16.9000,0.0000,NaN,inf,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.8700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.6894,NaN,0.0000,16.6894,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5210,0.5872,0.5889
16584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,-0.0000,NaN,NaN,NaN,1000.0173,8070000.0000,8517.9309,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.9217,1.0000,16.9217,0.0000,NaN,inf,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.8700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7206,NaN,0.0000,16.7206,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.2209,-0.2932,-0.2927
16583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,1000.0173,10700000.0000,2942.7876,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.9457,1.0000,16.9457,0.0000,NaN,inf,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.8750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.7494,NaN,0.0000,16.7494,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5810,0.3517,0.3523
16582,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,-0.0000,NaN,NaN,NaN,1000.0173,7730000.0000,-48083.4809,NaN,0.0000,NaN,0.0000,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.9225,0.0000,16.9225,1.0000,NaN,-inf,NaN,NaN,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1

In [ ]:
data9 = pd.concat([data7, self.data_out], axis = 1)
display_html(data9, 15)

(16590, 1270)


In [ ]:
data9[["Date","Close","DailyReturn","DailyReturn","DailyLogReturn","CumulativeReturn"]]

,Date,Close,DailyReturn,DailyReturn,DailyReturn,DailyReturn,DailyLogReturn,CumulativeReturn
16589,1950-01-03,16.6600,NaN,-96.5485,NaN,-96.5485,NaN,0.0000
16588,1950-01-04,16.8500,0.0114,1.1405,0.0114,1.1405,1.1340,1.1405
16587,1950-01-05,16.9300,0.0047,0.4748,0.0047,0.4748,0.4737,1.6206
16586,1950-01-06,16.9800,0.0030,0.2953,0.0030,0.2953,0.2949,1.9208
16585,1950-01-09,17.0800,0.0059,0.5889,0.0059,0.5889,0.5872,2.5210
...,...,...,...,...,...,...,...,...
4,2015-12-01,2102.6299,0.0107,1.0681,0.0107,1.0681,1.0624,12520.8276
3,2015-12-02,2079.5100,-0.0110,-1.0996,-0.0110,-1.0996,-1.1057,12382.0529
2,2015-12-03,2049.6201,-0.0144,-1.4374,-0.0144,-1.4374,-1.4478,12202.6418
1,2015-12-04,2091.6899,0.0205,2.0526,0.0205,2.0526,2.0318,12455.1617


In [ ]:
Kasztanki

NameError: name 'Kasztanki' is not defined

In [ ]:
# TRAIN TEST SPLIT

data['Date_split'] = np.where(data["Date"] > datetime(year=2015, month=4, day=1),1,0)

In [ ]:

# Create technical indicators

# https://www.investopedia.com/terms/s/sma.asp
data['SMA_5'] = ta.trend.sma_indicator(data['Close'], window=5)
data['SMA_5'] = data['SMA_5'].shift(periods=1)

data['SMA_10'] = ta.trend.sma_indicator(data['Close'], window=10)
data['SMA_10'] = data['SMA_10'].shift(periods=1)

data['SMA_30'] = ta.trend.sma_indicator(data['Close'], window=30)
data['SMA_30'] = data['SMA_30'].shift(periods=1)
# https://www.investopedia.com/terms/e/ema.asp
data['EMA_5'] = ta.trend.ema_indicator(data['Close'], window=5)
data['EMA_5'] = data['EMA_5'].shift(periods=1)

data['EMA_10'] = ta.trend.ema_indicator(data['Close'], window=10)
data['EMA_10'] = data['EMA_10'].shift(periods=1)

data['EMA_30'] = ta.trend.ema_indicator(data['Close'], window=30)
data['EMA_30'] = data['EMA_30'].shift(periods=1)








            Date  Open  High   Low  Close     Volume  Adj Close  Date_split  \
16589 1950-01-03 16.66 16.66 16.66  16.66 1260000.00      16.66           0   
16588 1950-01-04 16.85 16.85 16.85  16.85 1890000.00      16.85           0   
16587 1950-01-05 16.93 16.93 16.93  16.93 2550000.00      16.93           0   
16586 1950-01-06 16.98 16.98 16.98  16.98 2010000.00      16.98           0   
16585 1950-01-09 17.08 17.08 17.08  17.08 2520000.00      17.08           0   
16584 1950-01-10 17.03 17.03 17.03  17.03 2160000.00      17.03           0   
16583 1950-01-11 17.09 17.09 17.09  17.09 2630000.00      17.09           0   
16582 1950-01-12 16.76 16.76 16.76  16.76 2970000.00      16.76           0   
16581 1950-01-13 16.67 16.67 16.67  16.67 3330000.00      16.67           0   
16580 1950-01-16 16.72 16.72 16.72  16.72 1460000.00      16.72           0   
16579 1950-01-17 16.86 16.86 16.86  16.86 1790000.00      16.86           0   
16578 1950-01-18 16.85 16.85 16.85  16.85 1570000.00